# ADA - Homework 2

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context('notebook')
sns.set_style('whitegrid')

## Requesting and parsing data from IS-Academia

In [2]:
import requests
from bs4 import BeautifulSoup

I parse the first page to get all the option fields useful to make queries later 

In [3]:
r = requests.get('http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter?&ww_i_reportmodel=133685247')
soup = BeautifulSoup(r.text, 'html.parser')
options = {}
for option in soup.find_all('option'):
    options[option.parent.get('name')]=options.get(option.parent.get('name'),{})
    options[option.parent.get('name')][option.text]=option.get('value')
options

{'ww_x_HIVERETE': {'': 'null',
  "Semestre d'automne": '2936286',
  'Semestre de printemps': '2936295'},
 'ww_x_PERIODE_ACAD': {'': 'null',
  '2007-2008': '978181',
  '2008-2009': '978187',
  '2009-2010': '978195',
  '2010-2011': '39486325',
  '2011-2012': '123455150',
  '2012-2013': '123456101',
  '2013-2014': '213637754',
  '2014-2015': '213637922',
  '2015-2016': '213638028',
  '2016-2017': '355925344'},
 'ww_x_PERIODE_PEDAGO': {'': 'null',
  'Bachelor semestre 1': '249108',
  'Bachelor semestre 2': '249114',
  'Bachelor semestre 3': '942155',
  'Bachelor semestre 4': '942163',
  'Bachelor semestre 5': '942120',
  'Bachelor semestre 5b': '2226768',
  'Bachelor semestre 6': '942175',
  'Bachelor semestre 6b': '2226785',
  'Master semestre 1': '2230106',
  'Master semestre 2': '942192',
  'Master semestre 3': '2230128',
  'Master semestre 4': '2230140',
  'Mineur semestre 1': '2335667',
  'Mineur semestre 2': '2335676',
  'Mise à niveau': '2063602308',
  'Projet Master automne': '2491

In [4]:
def get_data(unite_acad, periode_acad, periode_pedago, hiverete):
    """ Get data with specific parameters.
    
        Args:
            unite_acad (str): categorical from options.ww_x_UNITE_ACAD.keys()
            periode_acad (str): categorical from options.ww_x_PERIODE_ACAD.keys()
            periode_pedago (str): categorical from options.ww_x_PERIODE_PEDAGO.keys()
            hiverete (str): categorical from options.ww_x_HIVERETE.keys()
            
        Returns:
            DataFrame: Pandas dataframe containing the requested data.
            
    """
    request_params = {
        'ww_x_GPS': '-1',
        'ww_i_reportModel': '133685247',
        'ww_i_reportModelXsl': '133685270',
        'ww_x_UNITE_ACAD': options['ww_x_UNITE_ACAD'][unite_acad],
        'ww_x_PERIODE_ACAD': options['ww_x_PERIODE_ACAD'][periode_acad],
        'ww_x_PERIODE_PEDAGO': options['ww_x_PERIODE_PEDAGO'][periode_pedago],
        'ww_x_HIVERETE': options['ww_x_HIVERETE'][hiverete]
    }
    
    r = requests.get('http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.html', params=request_params)

    soup = BeautifulSoup(r.text, 'html.parser')
    
    table_lines = soup.find_all('tr')
    columns = [th.text for th in table_lines[1].contents] if len(table_lines) > 1 else []
    nColumns = len(columns)
    
    data = {}
    for name in columns:
        data[name] = []
    
    for tr in table_lines[2:]:
        content = tr.contents
        if content[0].name == 'td' :
            for i in range(nColumns):
                data[columns[i]].append(content[i].text)
    
    result = pd.DataFrame(data=data)
    result['ww_x_UNITE_ACAD'] = unite_acad
    result['ww_x_PERIODE_ACAD'] = periode_acad
    result['ww_x_PERIODE_PEDAGO'] = periode_pedago
    result['ww_x_HIVERETE'] = hiverete
    return result

A small test for the `get_data` method

In [5]:
get_data(
    'Informatique',
    '2007-2008',
    'Bachelor semestre 1',
    "Semestre d'automne"
).head()

,Civilité,Ecole Echange,Filière opt.,Mineur,No Sciper,Nom Prénom,Orientation Bachelor,Orientation Master,Spécialisation,Statut,Type Echange,ww_x_UNITE_ACAD,ww_x_PERIODE_ACAD,ww_x_PERIODE_PEDAGO,ww_x_HIVERETE
0,Monsieur,,,,169569,Arévalo Christian,,,,Présent,,Informatique,2007-2008,Bachelor semestre 1,Semestre d'automne
1,Monsieur,,,,174905,Aubelle Flavien,,,,Présent,,Informatique,2007-2008,Bachelor semestre 1,Semestre d'automne
2,Monsieur,,,,173922,Badoud Morgan,,,,Présent,,Informatique,2007-2008,Bachelor semestre 1,Semestre d'automne
3,Monsieur,,,,179406,Baeriswyl Jonathan,,,,Présent,,Informatique,2007-2008,Bachelor semestre 1,Semestre d'automne
4,Monsieur,,,,179428,Barroco Michael,,,,Présent,,Informatique,2007-2008,Bachelor semestre 1,Semestre d'automne


In [6]:
def get_all_data(unite_acads, periode_acads, periode_pedagos, hiveretes):
    """ Get data with specific parameters.
    
        Args:
            unite_acads ([str]): array of categoricals from options.ww_x_UNITE_ACAD.keys()
            periode_acads ([str]): array of categoricals from options.ww_x_PERIODE_ACAD.keys()
            periode_pedagos ([str]): array of categoricals from options.ww_x_PERIODE_PEDAGO.keys()
            hiveretes ([str]): array of categoricals from options.ww_x_HIVERETE.keys()
            
        Returns:
            DataFrame: Pandas dataframe containing the requested data.
            
    """
    result = pd.DataFrame({})
    for unite_acad in unite_acads:
        for periode_acad in periode_acads:
            for periode_pedago in periode_pedagos:
                for hiverete in hiveretes:
                    print("_".join([unite_acad, periode_acad, periode_pedago, hiverete]))
                    result = pd.concat([result, get_data(unite_acad, periode_acad, periode_pedago, hiverete)])
    return result

## Question 1

### Importing the data

In [7]:
df_bachelor = get_all_data(
    ['Informatique'],
    ['2007-2008', '2008-2009', '2009-2010', '2010-2011', '2011-2012', '2012-2013', '2013-2014', '2014-2015', '2015-2016', '2016-2017'],
    ['Bachelor semestre 1', 'Bachelor semestre 2', 'Bachelor semestre 3', 'Bachelor semestre 4', 'Bachelor semestre 5', 'Bachelor semestre 6', 'Bachelor semestre 5b', 'Bachelor semestre 6b'],
    ["Semestre d'automne", 'Semestre de printemps']
)

Informatique_2007-2008_Bachelor semestre 1_Semestre d'automne
Informatique_2007-2008_Bachelor semestre 1_Semestre de printemps
Informatique_2007-2008_Bachelor semestre 2_Semestre d'automne
Informatique_2007-2008_Bachelor semestre 2_Semestre de printemps
Informatique_2007-2008_Bachelor semestre 3_Semestre d'automne
Informatique_2007-2008_Bachelor semestre 3_Semestre de printemps
Informatique_2007-2008_Bachelor semestre 4_Semestre d'automne
Informatique_2007-2008_Bachelor semestre 4_Semestre de printemps
Informatique_2007-2008_Bachelor semestre 5_Semestre d'automne
Informatique_2007-2008_Bachelor semestre 5_Semestre de printemps
Informatique_2007-2008_Bachelor semestre 6_Semestre d'automne
Informatique_2007-2008_Bachelor semestre 6_Semestre de printemps
Informatique_2007-2008_Bachelor semestre 5b_Semestre d'automne
Informatique_2007-2008_Bachelor semestre 5b_Semestre de printemps
Informatique_2007-2008_Bachelor semestre 6b_Semestre d'automne
Informatique_2007-2008_Bachelor semestre 6b_Se

In [8]:
print(df_bachelor.shape)
df_bachelor.head()

(5808, 15)


,Civilité,Ecole Echange,Filière opt.,Mineur,No Sciper,Nom Prénom,Orientation Bachelor,Orientation Master,Spécialisation,Statut,Type Echange,ww_x_HIVERETE,ww_x_PERIODE_ACAD,ww_x_PERIODE_PEDAGO,ww_x_UNITE_ACAD
0,Monsieur,,,,169569,Arévalo Christian,,,,Présent,,Semestre d'automne,2007-2008,Bachelor semestre 1,Informatique
1,Monsieur,,,,174905,Aubelle Flavien,,,,Présent,,Semestre d'automne,2007-2008,Bachelor semestre 1,Informatique
2,Monsieur,,,,173922,Badoud Morgan,,,,Présent,,Semestre d'automne,2007-2008,Bachelor semestre 1,Informatique
3,Monsieur,,,,179406,Baeriswyl Jonathan,,,,Présent,,Semestre d'automne,2007-2008,Bachelor semestre 1,Informatique
4,Monsieur,,,,179428,Barroco Michael,,,,Présent,,Semestre d'automne,2007-2008,Bachelor semestre 1,Informatique


### Exploring the data

In [9]:
print(df_bachelor.shape)
df_bachelor.head()

(5808, 15)


,Civilité,Ecole Echange,Filière opt.,Mineur,No Sciper,Nom Prénom,Orientation Bachelor,Orientation Master,Spécialisation,Statut,Type Echange,ww_x_HIVERETE,ww_x_PERIODE_ACAD,ww_x_PERIODE_PEDAGO,ww_x_UNITE_ACAD
0,Monsieur,,,,169569,Arévalo Christian,,,,Présent,,Semestre d'automne,2007-2008,Bachelor semestre 1,Informatique
1,Monsieur,,,,174905,Aubelle Flavien,,,,Présent,,Semestre d'automne,2007-2008,Bachelor semestre 1,Informatique
2,Monsieur,,,,173922,Badoud Morgan,,,,Présent,,Semestre d'automne,2007-2008,Bachelor semestre 1,Informatique
3,Monsieur,,,,179406,Baeriswyl Jonathan,,,,Présent,,Semestre d'automne,2007-2008,Bachelor semestre 1,Informatique
4,Monsieur,,,,179428,Barroco Michael,,,,Présent,,Semestre d'automne,2007-2008,Bachelor semestre 1,Informatique


5b ? 6b ?
These have no data! Youhou

In [10]:
print(df_bachelor[df_bachelor.ww_x_PERIODE_PEDAGO == 'Bachelor semestre 6b'].shape)
print(df_bachelor[df_bachelor.ww_x_PERIODE_PEDAGO == 'Bachelor semestre 5b'].shape)

(0, 15)
(0, 15)


Counting the students

In [11]:
print(len(df_bachelor['No Sciper'].unique()))

1481


## Filtering only students with a Bachelor semestre 1 & 6

In [12]:
df_bachelor_semestre_1 = df_bachelor[df_bachelor.ww_x_PERIODE_PEDAGO == 'Bachelor semestre 1']
df_bachelor_semestre_6 = df_bachelor[df_bachelor.ww_x_PERIODE_PEDAGO == 'Bachelor semestre 6']

In [13]:
df_bachelor_filtered = df_bachelor[df_bachelor['No Sciper'].isin(df_bachelor_semestre_1['No Sciper'].tolist())]
df_bachelor_filtered = df_bachelor_filtered[df_bachelor_filtered['No Sciper'].isin(df_bachelor_semestre_6['No Sciper'].tolist())]
print(df_bachelor_filtered.shape)
df_bachelor_filtered.head()

(2812, 15)


,Civilité,Ecole Echange,Filière opt.,Mineur,No Sciper,Nom Prénom,Orientation Bachelor,Orientation Master,Spécialisation,Statut,Type Echange,ww_x_HIVERETE,ww_x_PERIODE_ACAD,ww_x_PERIODE_PEDAGO,ww_x_UNITE_ACAD
0,Monsieur,,,,169569,Arévalo Christian,,,,Présent,,Semestre d'automne,2007-2008,Bachelor semestre 1,Informatique
1,Monsieur,,,,174905,Aubelle Flavien,,,,Présent,,Semestre d'automne,2007-2008,Bachelor semestre 1,Informatique
3,Monsieur,,,,179406,Baeriswyl Jonathan,,,,Présent,,Semestre d'automne,2007-2008,Bachelor semestre 1,Informatique
4,Monsieur,,,,179428,Barroco Michael,,,,Présent,,Semestre d'automne,2007-2008,Bachelor semestre 1,Informatique
7,Monsieur,,,,179449,Bindschaedler Vincent,,,,Présent,,Semestre d'automne,2007-2008,Bachelor semestre 1,Informatique


Counts the students

In [14]:
print(len(df_bachelor_filtered['No Sciper'].unique()))

397


## Compute average of number of months 
Method used: 
- group by sex and student 
- computes number_of_entries (corresponding on the number of semester studied) for each student
- computes average of (6*number_of_entries)




In [15]:
df_bachelor_grouped = df_bachelor_filtered[['Civilité', 'No Sciper', 'ww_x_PERIODE_PEDAGO']].groupby(['Civilité', 'No Sciper'])
print('Number of F: %d' % (df_bachelor_grouped.size()['Madame'].count()))
print('Number of M: %d' % (df_bachelor_grouped.size()['Monsieur'].count()))
print('F avg number of months: %f' % (df_bachelor_grouped.size()['Madame'].mean() * 6))
print('M avg number of months: %f' % (df_bachelor_grouped.size()['Monsieur'].mean() * 6))

Number of F: 29
Number of M: 368
F avg number of months: 40.758621
M avg number of months: 42.635870


### Statistical significance

In [16]:
from scipy.stats import ttest_ind

In [17]:
f = [6 * x for x in df_bachelor_grouped.size()['Madame'].tolist()]
m = [6 * x for x in df_bachelor_grouped.size()['Monsieur'].tolist()]

In [18]:
ttest_ind(f, m)

Ttest_indResult(statistic=-1.0643000334248713, pvalue=0.28784297465161934)

## Question 2

### Importing

In [19]:
df_master = get_all_data(
    ['Informatique'],
    ['2007-2008', '2008-2009', '2009-2010', '2010-2011', '2011-2012', '2012-2013', '2013-2014', '2014-2015', '2015-2016', '2016-2017'],
    ['Master semestre 1', 'Master semestre 2', 'Master semestre 3', 'Master semestre 4', 'Mineur semestre 1', 'Mineur semestre 2', 'Projet Master automne', 'Projet Master printemps'],
    ["Semestre d'automne", 'Semestre de printemps']
)

Informatique_2007-2008_Master semestre 1_Semestre d'automne
Informatique_2007-2008_Master semestre 1_Semestre de printemps
Informatique_2007-2008_Master semestre 2_Semestre d'automne
Informatique_2007-2008_Master semestre 2_Semestre de printemps
Informatique_2007-2008_Master semestre 3_Semestre d'automne
Informatique_2007-2008_Master semestre 3_Semestre de printemps
Informatique_2007-2008_Master semestre 4_Semestre d'automne
Informatique_2007-2008_Master semestre 4_Semestre de printemps
Informatique_2007-2008_Mineur semestre 1_Semestre d'automne
Informatique_2007-2008_Mineur semestre 1_Semestre de printemps
Informatique_2007-2008_Mineur semestre 2_Semestre d'automne
Informatique_2007-2008_Mineur semestre 2_Semestre de printemps
Informatique_2007-2008_Projet Master automne_Semestre d'automne
Informatique_2007-2008_Projet Master automne_Semestre de printemps
Informatique_2007-2008_Projet Master printemps_Semestre d'automne
Informatique_2007-2008_Projet Master printemps_Semestre de printe

In [20]:
print(df_master.shape)
df_master.head()

(2844, 15)


,Civilité,Ecole Echange,Filière opt.,Mineur,No Sciper,Nom Prénom,Orientation Bachelor,Orientation Master,Spécialisation,Statut,Type Echange,ww_x_HIVERETE,ww_x_PERIODE_ACAD,ww_x_PERIODE_PEDAGO,ww_x_UNITE_ACAD
0,Monsieur,,,,153066,Aeberhard François-Xavier,,,,Présent,,Semestre d'automne,2007-2008,Master semestre 1,Informatique
1,Madame,,,,180027,Agarwal Megha,,,,Présent,,Semestre d'automne,2007-2008,Master semestre 1,Informatique
2,Monsieur,,,,152232,Anagnostaras David,,,,Présent,,Semestre d'automne,2007-2008,Master semestre 1,Informatique
3,Monsieur,,,,177395,Auroux Damien,,,,Présent,,Semestre d'automne,2007-2008,Master semestre 1,Informatique
4,Monsieur,,,,161970,Awalebo Joseph,,,,Présent,,Semestre d'automne,2007-2008,Master semestre 1,Informatique


### Exploring

About the field Mineur

In [21]:
df_master['Mineur'].value_counts()

                                                        2610
Mineur en Management, technologie et entrepreneuriat     114
Information security minor                                31
Mineur en Ingénierie financière                           25
Mineur en Biocomputing                                    24
Area and Cultural Studies minor                           10
Mineur en Technologies spatiales                           8
Mineur en Mathématiques                                    7
Mineur en Neurosciences computationnelles                  7
Mineur en Etudes asiatiques contemporaines                 3
Mineur en Microtechnique                                   2
Mineur en Neuroprosthétiques                               2
Mineur STAS Chine                                          1
Name: Mineur, dtype: int64

In [22]:
df_master_mineur = df_master[df_master.Mineur != ""]
(df_master['No Sciper'].unique().size, df_master_mineur['No Sciper'].unique().size)

(977, 84)

In [23]:
df_master['DidMineur'] = df_master['No Sciper'].isin(df_master_mineur['No Sciper'].tolist())
df_master.head()

,Civilité,Ecole Echange,Filière opt.,Mineur,No Sciper,Nom Prénom,Orientation Bachelor,Orientation Master,Spécialisation,Statut,Type Echange,ww_x_HIVERETE,ww_x_PERIODE_ACAD,ww_x_PERIODE_PEDAGO,ww_x_UNITE_ACAD,DidMineur
0,Monsieur,,,,153066,Aeberhard François-Xavier,,,,Présent,,Semestre d'automne,2007-2008,Master semestre 1,Informatique,False
1,Madame,,,,180027,Agarwal Megha,,,,Présent,,Semestre d'automne,2007-2008,Master semestre 1,Informatique,False
2,Monsieur,,,,152232,Anagnostaras David,,,,Présent,,Semestre d'automne,2007-2008,Master semestre 1,Informatique,True
3,Monsieur,,,,177395,Auroux Damien,,,,Présent,,Semestre d'automne,2007-2008,Master semestre 1,Informatique,False
4,Monsieur,,,,161970,Awalebo Joseph,,,,Présent,,Semestre d'automne,2007-2008,Master semestre 1,Informatique,False


About the field `ww_x_PERIODE_PEDAGO` and the options 'Mineur semestre 1' and 'Mineur semestre 2'

In [24]:
print(df_master[df_master.ww_x_PERIODE_PEDAGO == 'Mineur semestre 1'].shape)
print(df_master[df_master.ww_x_PERIODE_PEDAGO == 'Mineur semestre 2'].shape)

(0, 16)
(0, 16)


### Filtering

We decide to consider that a student completes the master by doing his/her Master Project

In [25]:
df_master_pdm = df_master[df_master.ww_x_PERIODE_PEDAGO.isin(['Projet Master automne','Projet Master printemps'])]

In [26]:
df_master_filtered = df_master[df_master['No Sciper'].isin(df_master_pdm['No Sciper'].tolist())]
df_master_filtered.head()

,Civilité,Ecole Echange,Filière opt.,Mineur,No Sciper,Nom Prénom,Orientation Bachelor,Orientation Master,Spécialisation,Statut,Type Echange,ww_x_HIVERETE,ww_x_PERIODE_ACAD,ww_x_PERIODE_PEDAGO,ww_x_UNITE_ACAD,DidMineur
1,Madame,,,,180027,Agarwal Megha,,,,Présent,,Semestre d'automne,2007-2008,Master semestre 1,Informatique,False
8,Madame,,,,154573,Benabdallah Zeineb,,,,Présent,,Semestre d'automne,2007-2008,Master semestre 1,Informatique,False
10,Monsieur,,,,172687,Billaud Joël,,,,Présent,,Semestre d'automne,2007-2008,Master semestre 1,Informatique,False
13,Monsieur,,,,180072,Campora Simone,,,Internet computing,Présent,,Semestre d'automne,2007-2008,Master semestre 1,Informatique,False
16,Monsieur,,,,160225,Cassata Alexandre,,,,Présent,,Semestre d'automne,2007-2008,Master semestre 1,Informatique,False


In [27]:
df_master_grouped = df_master_filtered[['Civilité', 'No Sciper', 'ww_x_PERIODE_PEDAGO']].groupby(['Civilité', 'No Sciper'])
print('Number of F: %d' % (df_master_grouped.size()['Madame'].count()))
print('Number of M: %d' % (df_master_grouped.size()['Monsieur'].count()))
print('F avg number of months: %f' % (df_master_grouped.size()['Madame'].mean() * 6))
print('M avg number of months: %f' % (df_master_grouped.size()['Monsieur'].mean() * 6))

number of F: 13
number of M: 108
F avg number of months: 24.923077
M avg number of months: 24.888889


In [28]:
df_master_grouped.size()['Madame'].tolist()

[6, 3, 3, 4, 4, 3, 5, 3, 3, 5, 5, 5, 5]

Let's explore the time difference by also grouping by mineur

In [29]:
df_master_grouped_with_mineur = df_master_filtered[['No Sciper','ww_x_PERIODE_PEDAGO','DidMineur']].groupby(['DidMineur','No Sciper'])

In [30]:
print('number of WITH: %d' % (df_master_grouped_with_mineur.size()[True].count()))
print('number of WITHOUT: %d' % (df_master_grouped_with_mineur.size()[False].count()))
print('WITH avg number of months: %f' % (df_master_grouped_with_mineur.size()[True].mean()*6))
print('WITHOUT avg number of months: %f' % (df_master_grouped_with_mineur.size()[False].mean()*6))

number of WITH: 8
number of WITHOUT: 113
WITH avg number of months: 28.500000
WITHOUT avg number of months: 24.637168


### Statistical significance

same as previous section

In [31]:
f = [6*x for x in df_master_grouped.size()['Madame'].tolist()]
m = [6*x for x in df_master_grouped.size()['Monsieur'].tolist()]
ttest_ind(f, m)

Ttest_indResult(statistic=0.017212195975248677, pvalue=0.98629615785433222)